In [1]:
%load_ext pretty_jupyter

In [2]:
# Import Notebook Libraries
from plotly.offline import plot
import pandas as pd
from IPython.display import display, HTML
import plotly.express as px
import plotly.graph_objects as go
import plotly.io
from pretty_jupyter.helpers import matplotlib_fig_to_html
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
from IPython.display import HTML

In [3]:
plotly.io.renderers.default = 'notebook_connected'

In [7]:
%%jmd


![NoFace](noface.png)
**Image Source:** *Studio Ghibli* **Movie:** *"Spirited Away"*

# Introduction

Studio Ghibli stands as a paragon of animation and storytelling, distinguished by its ability to portray complex emotional narratives with breathtakingly beautiful visuals. Known for its artistic vision, Ghibli has not only pioneered numerous animation techniques but also set new benchmarks for narrative depth in animation. This project aims to analyze the success of various genres within Studio Ghibli's rich catalog, focusing on which genres have been most successful. By examining key aspects such as financial performance, directorial influence, budgets, revenues, and film durations, the analysis seeks to highlight the unique attributes and production dynamics that have contributed significantly to Studio Ghibli's enduring success and cultural impact.


## Project Overview

This project conducts an in-depth analysis of Studio Ghibli films, focusing on the success of different genres within the studio’s film catalog. By examining elements like financial performance, budgets, revenues, directorial influence, and film durations, the study aims to determine which genres have been most effective in contributing to the studio's financial success and audience reach. The objective is to provide insights that can inform future film projects and marketing strategies, ensuring the continued success and resonance of Studio Ghibli’s productions with audiences worldwide.

## Business Questions

1. **Which genre combinations are most frequently produced by Studio Ghibli, and what does this indicate about viewer preferences?**
   - This question explores the production frequency to infer genre popularity and audience trends.

2. **Which genre combinations yield the highest financial returns for Studio Ghibli?**
   - Identifying the genres that generate the most revenue can suggest where future financial investments could be most profitable.

3. **How do film durations vary across different genres, and what might this suggest about audience engagement?**
   - Understanding the correlation between film length and genre can help tailor future productions to better meet audience expectations.

4. **What is the impact of different directors on the financial and critical success of various genres?**
   - Analyzing directorial influence on genre success can guide decisions about director assignments to optimize film performance.

### Setup
#### Import Require Libraries
```python
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import tableauserverclient as TSC
```
#### Load Data
```python
data = pd.read_csv('Studio_Ghibli.csv')
```



![NoFace](noface.png)
**Image Source:** *Studio Ghibli* **Movie:** *"Spirited Away"*

# Introduction

Studio Ghibli stands as a paragon of animation and storytelling, distinguished by its ability to portray complex emotional narratives with breathtakingly beautiful visuals. Known for its artistic vision, Ghibli has not only pioneered numerous animation techniques but also set new benchmarks for narrative depth in animation. This project aims to analyze the success of various genres within Studio Ghibli's rich catalog, focusing on which genres have been most successful. By examining key aspects such as financial performance, directorial influence, budgets, revenues, and film durations, the analysis seeks to highlight the unique attributes and production dynamics that have contributed significantly to Studio Ghibli's enduring success and cultural impact.


## Project Overview

This project conducts an in-depth analysis of Studio Ghibli films, focusing on the success of different genres within the studio’s film catalog. By examining elements like financial performance, budgets, revenues, directorial influence, and film durations, the study aims to determine which genres have been most effective in contributing to the studio's financial success and audience reach. The objective is to provide insights that can inform future film projects and marketing strategies, ensuring the continued success and resonance of Studio Ghibli’s productions with audiences worldwide.

## Business Questions

1. **Which genre combinations are most frequently produced by Studio Ghibli, and what does this indicate about viewer preferences?**
   - This question explores the production frequency to infer genre popularity and audience trends.

2. **Which genre combinations yield the highest financial returns for Studio Ghibli?**
   - Identifying the genres that generate the most revenue can suggest where future financial investments could be most profitable.

3. **How do film durations vary across different genres, and what might this suggest about audience engagement?**
   - Understanding the correlation between film length and genre can help tailor future productions to better meet audience expectations.

4. **What is the impact of different directors on the financial and critical success of various genres?**
   - Analyzing directorial influence on genre success can guide decisions about director assignments to optimize film performance.

### Setup
#### Import Require Libraries
```python
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import tableauserverclient as TSC
```
#### Load Data
```python
data = pd.read_csv('Studio_Ghibli.csv')
```

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import tableauserverclient as TSC

In [7]:
data = pd.read_csv('Studio_Ghibli.csv')
data.head()

,Name,Year,Director,Screenplay,Budget,Revenue,Genre 1,Genre 2,Genre 3,Duration
0,When Marnie Was There\n (2014),2014,Hiromasa Yonebayashi,Joan G. Robinson,$1150000000.00,$34949567.00,Animation,Drama,NaN,1h 43m
1,The Tale of The Princess Kaguya\n (2013),2013,Isao Takahata,Riko Sakaguchi,$49300000.00,$24366656.00,Animation,Drama,Fantasy,2h 17m
2,The Wind Rises\n (2013),2013,Hayao Miyazaki,Tatsuo Hori,$30000000.00,$117932401.00,Drama,Animation,Romance,2h 6m
3,From Up on Poppy Hill\n (2011),2011,Goro Miyazaki,Hayao Miyazaki,$22000000.00,$61037844.00,Animation,Drama,NaN,1h 31m
4,The Secret World of Arrietty\n (2010),2010,Hiromasa Yonebayashi,Mary Norton,$23000000.00,$149480483.00,Fantasy,Animation,Family,1h 34m


In [8]:
%%jmd

___

# Exploratory Data Analysis (EDA)

## Data Cleaning
1. Remove the suffix "\n(20__)" in the `Name` column.
```python
# Remove '\n' and year suffix from the "Name" column
data['Name'] = data['Name'].str.replace(r'\s*\(\d+\)', '', regex=True)
```

2. Convert `Revenue` and `Budget` to numerical values.
```python
data['Revenue'] = data['Revenue'].replace('[\$,]', '', regex=True).astype(float)
data['Budget'] = data['Budget'].replace('[\$,]', '', regex=True).astype(float)
```

3. Handle any missing values in the `Genre 3` and `Screenplay` columns.
```python
# Checking for missing values in genre columns
missing_values = data[['Genre 1', 'Genre 2', 'Genre 3', 'Screenplay']].isnull().sum()
```

4. Verify and possibly format the `Year` and `Duration` for consistent analysis.
```python
# Function to convert duration formatted as 'Xh Ym' to total minutes
def convert_duration_to_minutes(duration):
    hours, minutes = duration.split('h ')
    minutes = minutes.strip('m')  # Remove the 'm' from minutes
    total_minutes = int(hours) * 60 + int(minutes)
    return total_minutes
```


___

# Exploratory Data Analysis (EDA)

## Data Cleaning
1. Remove the suffix "\n(20__)" in the `Name` column.
```python
# Remove '\n' and year suffix from the "Name" column
data['Name'] = data['Name'].str.replace(r'\s*\(\d+\)', '', regex=True)
```

2. Convert `Revenue` and `Budget` to numerical values.
```python
data['Revenue'] = data['Revenue'].replace('[\$,]', '', regex=True).astype(float)
data['Budget'] = data['Budget'].replace('[\$,]', '', regex=True).astype(float)
```

3. Handle any missing values in the `Genre 3` and `Screenplay` columns.
```python
# Checking for missing values in genre columns
missing_values = data[['Genre 1', 'Genre 2', 'Genre 3', 'Screenplay']].isnull().sum()
```

4. Verify and possibly format the `Year` and `Duration` for consistent analysis.
```python
# Function to convert duration formatted as 'Xh Ym' to total minutes
def convert_duration_to_minutes(duration):
    hours, minutes = duration.split('h ')
    minutes = minutes.strip('m')  # Remove the 'm' from minutes
    total_minutes = int(hours) * 60 + int(minutes)
    return total_minutes
```

In [9]:
# Remove '\n' and year suffix from the "Name" column
data['Name'] = data['Name'].str.replace(r'\s*\(\d+\)', '', regex=True)

# 2. Convert 'Revenue' and 'Budget' to numerical values by removing non-numeric characters and converting to float
data['Revenue'] = data['Revenue'].replace('[\$,]', '', regex=True).astype(float)
data['Budget'] = data['Budget'].replace('[\$,]', '', regex=True).astype(float)

# 3. Handling missing values for genres
# Checking for missing values in genre columns
missing_values = data[['Genre 1', 'Genre 2', 'Genre 3', 'Screenplay']].isnull().sum()

# 4. Verify and format 'Year' and 'Duration'
# Ensuring 'Year' is an integer
data['Year'] = data['Year'].astype(int)

# Function to convert duration formatted as 'Xh Ym' to total minutes
def convert_duration_to_minutes(duration):
    hours, minutes = duration.split('h ')
    minutes = minutes.strip('m')  # Remove the 'm' from minutes
    total_minutes = int(hours) * 60 + int(minutes)
    return total_minutes

# Apply this function to each entry in the 'Duration' column
data['Duration_in_minutes'] = data['Duration'].apply(convert_duration_to_minutes)

missing_values, data.head()

(Genre 1       0
 Genre 2       0
 Genre 3       4
 Screenplay    9
 dtype: int64,
                               Name  Year              Director  \
 0            When Marnie Was There  2014  Hiromasa Yonebayashi   
 1  The Tale of The Princess Kaguya  2013         Isao Takahata   
 2                   The Wind Rises  2013        Hayao Miyazaki   
 3            From Up on Poppy Hill  2011         Goro Miyazaki   
 4     The Secret World of Arrietty  2010  Hiromasa Yonebayashi   
 
          Screenplay        Budget      Revenue    Genre 1    Genre 2  Genre 3  \
 0  Joan G. Robinson  1.150000e+09   34949567.0  Animation      Drama      NaN   
 1    Riko Sakaguchi  4.930000e+07   24366656.0  Animation      Drama  Fantasy   
 2       Tatsuo Hori  3.000000e+07  117932401.0      Drama  Animation  Romance   
 3    Hayao Miyazaki  2.200000e+07   61037844.0  Animation      Drama      NaN   
 4       Mary Norton  2.300000e+07  149480483.0    Fantasy  Animation   Family   
 
   Duration  Durati

In [10]:
%%jmd

- Missing values in `Genre 3` were identified. There are 4 entries missing this information. Since the missing values are due to some films simply not having a third genre, we will leave them as NaN to ensure data integrity.
- `Genre 1` and `Genre 2` do not have missing values.
- `Screenplay` has 9 missing entries. After futher research, it appears that the missing entries in this column indicate the the Director also wrote the screenplay for the film. So we will fill the missing values in `Screenplay` with the values from the corresponding rows in the `Director` columm.
```python
# Fill missing values in 'Screenplay' with values from 'Director' column
data['Screenplay'].fillna(data['Director'], inplace=True)
```


- Missing values in `Genre 3` were identified. There are 4 entries missing this information. Since the missing values are due to some films simply not having a third genre, we will leave them as NaN to ensure data integrity.
- `Genre 1` and `Genre 2` do not have missing values.
- `Screenplay` has 9 missing entries. After futher research, it appears that the missing entries in this column indicate the the Director also wrote the screenplay for the film. So we will fill the missing values in `Screenplay` with the values from the corresponding rows in the `Director` columm.
```python
# Fill missing values in 'Screenplay' with values from 'Director' column
data['Screenplay'].fillna(data['Director'], inplace=True)
```

In [11]:
# Fill missing values in 'Screenplay' with values from 'Director' column
data['Screenplay'].fillna(data['Director'], inplace=True)

In [90]:
%%jmd

## Data Exploration
Summarize the data to understand the distribution of genres, revenues, and other metrics.

### Summary Statistics
Providing an overview of the central tendency, dispersion, and other key metrics of the numerical columns. 
```python
# Summary statistics for numerical columns
summary_stats = data[['Revenue', 'Budget', 'Year', 'Duration_in_minutes']].describe()
```


## Data Exploration
Summarize the data to understand the distribution of genres, revenues, and other metrics.

### Summary Statistics
Providing an overview of the central tendency, dispersion, and other key metrics of the numerical columns. 
```python
# Summary statistics for numerical columns
summary_stats = data[['Revenue', 'Budget', 'Year', 'Duration_in_minutes']].describe()
```

In [13]:
# Summary statistics for numerical columns
summary_stats = data[['Revenue', 'Budget', 'Year', 'Duration_in_minutes']].describe()
print(summary_stats)

            Revenue        Budget         Year  Duration_in_minutes
count  2.300000e+01  2.300000e+01    23.000000            23.000000
mean   2.748218e+08  6.918696e+07  2000.086957           107.869565
std    9.183466e+08  2.365557e+08    10.697974            18.149141
min    1.000000e+05  1.000000e+06  1984.000000            72.000000
25%    1.479770e+07  5.950000e+06  1991.500000            93.500000
50%    5.401637e+07  2.000000e+07  1999.000000           111.000000
75%    1.670000e+08  2.375000e+07  2009.000000           121.500000
max    4.470000e+09  1.150000e+09  2023.000000           137.000000


In [91]:
%%jmd

- **Revenue**: 
  - The average revenue generated by Studio Ghibli films is approximately 274.8 million.
  - The revenue varies widely, with a standard deviation of approximately 918.35 million.
  - The minimum revenue is 100k, while the maximum is approximately 4.47 billion, suggesting a wide range of financial success among the films.

- **Budget**:
  - The average production budget for Studio Ghibli films is approximately 69.2 million.
  - Similar to revenue, the budget also exhibits significant variation, with a standard deviation of approximately 236.56 million.
  - The minimum budget is 1 million, while the maximum is 1.15 billion.

- **Year**:
  - The films in the dataset were released between 1984 and 2023, spanning nearly four decades.
  - The median release year is 1999, indicating that half of the films were released before and after this year.

- **Duration in Minutes**:
  - The average duration of Studio Ghibli films is approximately 107.9 minutes.
  - The durations range from 72 minutes to 137 minutes, with a standard deviation of approximately 18.15 minutes, indicating some variation in film lengths.

### Genre Distribution
Examining the distribution of genres in the dataset by counting the occurrences of each genre in the `Genre 1`, `Genre 2`, and `Genre 3` columns.
```python
# Count the occurrences of each genre
genre_counts = data[['Genre 1', 'Genre 2', 'Genre 3']].stack().value_counts()
```


- **Revenue**: 
  - The average revenue generated by Studio Ghibli films is approximately 274.8 million.
  - The revenue varies widely, with a standard deviation of approximately 918.35 million.
  - The minimum revenue is 100k, while the maximum is approximately 4.47 billion, suggesting a wide range of financial success among the films.

- **Budget**:
  - The average production budget for Studio Ghibli films is approximately 69.2 million.
  - Similar to revenue, the budget also exhibits significant variation, with a standard deviation of approximately 236.56 million.
  - The minimum budget is 1 million, while the maximum is 1.15 billion.

- **Year**:
  - The films in the dataset were released between 1984 and 2023, spanning nearly four decades.
  - The median release year is 1999, indicating that half of the films were released before and after this year.

- **Duration in Minutes**:
  - The average duration of Studio Ghibli films is approximately 107.9 minutes.
  - The durations range from 72 minutes to 137 minutes, with a standard deviation of approximately 18.15 minutes, indicating some variation in film lengths.

### Genre Distribution
Examining the distribution of genres in the dataset by counting the occurrences of each genre in the `Genre 1`, `Genre 2`, and `Genre 3` columns.
```python
# Count the occurrences of each genre
genre_counts = data[['Genre 1', 'Genre 2', 'Genre 3']].stack().value_counts()
```

In [15]:
# Count the occurrences of each genre
genre_counts = data[['Genre 1', 'Genre 2', 'Genre 3']].stack().value_counts()
print(genre_counts)

Animation    22
Fantasy      14
Drama         8
Family        8
Adventure     8
Romance       3
War           1
Comedy        1
Name: count, dtype: int64


In [16]:
%%jmd

- **Animation**: 
  - Animation is the most common genre, appearing in 22 out of 23 films in the dataset.

- **Fantasy**, **Drama**, **Family**, and **Adventure**:
  - These genres are also popular, each appearing in 8 films.

- **Romance**, **War**, and **Comedy**:
  - These genres are less common, appearing in only a few films each.
  - Romance and War appear in 3 films each, while Comedy appears in 1 film.


- **Animation**: 
  - Animation is the most common genre, appearing in 22 out of 23 films in the dataset.

- **Fantasy**, **Drama**, **Family**, and **Adventure**:
  - These genres are also popular, each appearing in 8 films.

- **Romance**, **War**, and **Comedy**:
  - These genres are less common, appearing in only a few films each.
  - Romance and War appear in 3 films each, while Comedy appears in 1 film.

In [92]:
%%jmd

### Budget & Revenue Distribution by Genre
Exploring the distribution of revenue across different genres by calculating summary statistics for revenue within each genre.
```python
# Summary statistics for budget by genre
budget_by_genre = data.groupby('Genre 1')['Budget'].describe()
# Summary statistics for revenue by genre
revenue_by_genre = data.groupby('Genre 1')['Revenue'].describe()
```


### Budget & Revenue Distribution by Genre
Exploring the distribution of revenue across different genres by calculating summary statistics for revenue within each genre.
```python
# Summary statistics for budget by genre
budget_by_genre = data.groupby('Genre 1')['Budget'].describe()
# Summary statistics for revenue by genre
revenue_by_genre = data.groupby('Genre 1')['Revenue'].describe()
```

In [18]:
# Summary statistics for budget by genre
budget_by_genre = data.groupby('Genre 1')['Budget'].describe()
print(budget_by_genre)

# Summary statistics for revenue by genre
revenue_by_genre = data.groupby('Genre 1')['Revenue'].describe()
print(revenue_by_genre)

           count         mean           std         min         25%  \
Genre 1                                                               
Adventure    5.0   11900000.0  9.977475e+06   1000000.0   3000000.0   
Animation   12.0  111825000.0  3.272363e+08   2500000.0   6425000.0   
Drama        1.0   30000000.0           NaN  30000000.0  30000000.0   
Family       1.0    9200000.0           NaN   9200000.0   9200000.0   
Fantasy      4.0   37675000.0  4.258743e+07   3700000.0  18175000.0   

                  50%         75%           max  
Genre 1                                          
Adventure  12000000.0  20000000.0  2.350000e+07  
Animation  19750000.0  25000000.0  1.150000e+09  
Drama      30000000.0  30000000.0  3.000000e+07  
Family      9200000.0   9200000.0  9.200000e+06  
Fantasy    23500000.0  43000000.0  1.000000e+08  
           count         mean           std          min           25%  \
Genre 1                                                                  
Adve

In [19]:
%%jmd

#### Budget by Genre:
- **Animation**: 
  - Mean budget: 111.8 million.
  - Standard deviation: 327.24 million.
  - The range of budgets is wide, from 2.5 million to 1.15 billion.

- **Fantasy**: 
  - Mean budget: 37.68 million.
  - Standard deviation: 42.59 million.
  - The range of budgets is from 3.7 million to 100 million.

- **Adventure**:
  - Mean budget: 11.9 million.
  - Standard deviation: 9.98 million.
  - The range of budgets is from 1 million to 23.5 million.

- **Drama** and **Family**: 
  - Each has only one entry in the dataset, with budgets of 30 million and 9.2 million, respectively.

#### Revenue by Genre:
- **Animation**: 
  - Mean revenue: 72.82 million.
  - Standard deviation: 91.6 million.
  - The range of revenues is wide, from 100,000 to 274.93 million.

- **Fantasy**: 
  - Mean revenue: 148.38 million.
  - Standard deviation: 80.76 million.
  - The range of revenues is from 41 million to 236.05 million.

- **Adventure**:
  - Mean revenue: 940.31 million.
  - Standard deviation: 1.97 billion.
  - The range of revenues is from 3.3 million to 4.47 billion.

- **Drama** and **Family**: 
  - Each has only one entry in the dataset, with revenues of 117.93 million and 34.1 million, respectively.
___




#### Budget by Genre:
- **Animation**: 
  - Mean budget: 111.8 million.
  - Standard deviation: 327.24 million.
  - The range of budgets is wide, from 2.5 million to 1.15 billion.

- **Fantasy**: 
  - Mean budget: 37.68 million.
  - Standard deviation: 42.59 million.
  - The range of budgets is from 3.7 million to 100 million.

- **Adventure**:
  - Mean budget: 11.9 million.
  - Standard deviation: 9.98 million.
  - The range of budgets is from 1 million to 23.5 million.

- **Drama** and **Family**: 
  - Each has only one entry in the dataset, with budgets of 30 million and 9.2 million, respectively.

#### Revenue by Genre:
- **Animation**: 
  - Mean revenue: 72.82 million.
  - Standard deviation: 91.6 million.
  - The range of revenues is wide, from 100,000 to 274.93 million.

- **Fantasy**: 
  - Mean revenue: 148.38 million.
  - Standard deviation: 80.76 million.
  - The range of revenues is from 41 million to 236.05 million.

- **Adventure**:
  - Mean revenue: 940.31 million.
  - Standard deviation: 1.97 billion.
  - The range of revenues is from 3.3 million to 4.47 billion.

- **Drama** and **Family**: 
  - Each has only one entry in the dataset, with revenues of 117.93 million and 34.1 million, respectively.
___

In [20]:
%%jmd

# Insights

The findings from the Exploratory Data Analysis (EDA) provide valuable insights. By examining revenue, budget, genre distribution, and other metrics, we can draw conclusions that can inform future decisions regarding genre selection and resource allocation in Studio Ghibli's filmmaking endeavors.

Key insights include:

1. **Revenue and Budget Analysis**:
   - Studio Ghibli films vary widely in revenue, from 100,000 to 4.47 billion, indicating diverse financial success.
   - Adventure films stand out with the highest mean revenue of 940.31 million, followed by Fantasy (148.38 million) and Animation (72.82 million).
   - Similarly, Adventure films have the highest mean budget (11.9 million), suggesting substantial investments for potentially high returns.

2. **Genre Distribution**:
   - Animation is the most prevalent genre, appearing in 22 out of 23 films, highlighting its popularity.
   - Fantasy, Drama, Family, and Adventure are also prominent, each appearing in 8 films, showcasing genre diversity.

3. **Budget and Revenue Distribution by Genre**:
   - Adventure films lead in revenue, with a mean of 940.31 million, followed by Fantasy (148.38 million) and Animation (72.82 million).
   - Adventure films also have the highest mean budget (11.9 million), indicating significant investment.
   
4. **Year and Duration Analysis**:
   - Studio Ghibli's filmography spans nearly four decades, from 1984 to 2023, indicating longevity in the industry.
   - The median release year is 1999, suggesting a balanced distribution of films over time.
   - Films average approximately 107.9 minutes in duration, reflecting varied storytelling approaches.

In summary, Adventure films emerge as the most lucrative within Studio Ghibli's filmography, followed by Fantasy and Animation. These insights inform future decisions on genre selection and resource allocation, aiming to balance artistic vision with commercial success.

___


# Insights

The findings from the Exploratory Data Analysis (EDA) provide valuable insights. By examining revenue, budget, genre distribution, and other metrics, we can draw conclusions that can inform future decisions regarding genre selection and resource allocation in Studio Ghibli's filmmaking endeavors.

Key insights include:

1. **Revenue and Budget Analysis**:
   - Studio Ghibli films vary widely in revenue, from 100,000 to 4.47 billion, indicating diverse financial success.
   - Adventure films stand out with the highest mean revenue of 940.31 million, followed by Fantasy (148.38 million) and Animation (72.82 million).
   - Similarly, Adventure films have the highest mean budget (11.9 million), suggesting substantial investments for potentially high returns.

2. **Genre Distribution**:
   - Animation is the most prevalent genre, appearing in 22 out of 23 films, highlighting its popularity.
   - Fantasy, Drama, Family, and Adventure are also prominent, each appearing in 8 films, showcasing genre diversity.

3. **Budget and Revenue Distribution by Genre**:
   - Adventure films lead in revenue, with a mean of 940.31 million, followed by Fantasy (148.38 million) and Animation (72.82 million).
   - Adventure films also have the highest mean budget (11.9 million), indicating significant investment.
   
4. **Year and Duration Analysis**:
   - Studio Ghibli's filmography spans nearly four decades, from 1984 to 2023, indicating longevity in the industry.
   - The median release year is 1999, suggesting a balanced distribution of films over time.
   - Films average approximately 107.9 minutes in duration, reflecting varied storytelling approaches.

In summary, Adventure films emerge as the most lucrative within Studio Ghibli's filmography, followed by Fantasy and Animation. These insights inform future decisions on genre selection and resource allocation, aiming to balance artistic vision with commercial success.

___

In [21]:
data.to_csv('Studio_Ghibli_Cleaned.csv', index=False)

In [22]:
%%jmd

# Data Preparation for Tableau

### Distribution of Primary and Seconday Genres

Showcasing the distribution of genre combinations to highlight which specific combinations of genres (e.g., Animation-Drama, Animation-Fantasy) are most common and potentially most successful.
Since not all films have a third genre, we will use only `Genre 1` and `Genre 2`.
```python
# Create a new column combining the genre information
ghibli_data['Combined_Genres'] = ghibli_data[['Genre 1', 'Genre 2']].apply(
    lambda row: '-'.join(row.dropna().astype(str)), axis=1)

# Count of each genre combination in the new Combined_Genres column 
genre_combination_distribution = ghibli_data['Combined_Genres'].value_counts().reset_index()
genre_combination_distribution.columns = ['Combined_Genres', 'Film_Count']
```


# Data Preparation for Tableau

### Distribution of Primary and Seconday Genres

Showcasing the distribution of genre combinations to highlight which specific combinations of genres (e.g., Animation-Drama, Animation-Fantasy) are most common and potentially most successful.
Since not all films have a third genre, we will use only `Genre 1` and `Genre 2`.
```python
# Create a new column combining the genre information
ghibli_data['Combined_Genres'] = ghibli_data[['Genre 1', 'Genre 2']].apply(
    lambda row: '-'.join(row.dropna().astype(str)), axis=1)

# Count of each genre combination in the new Combined_Genres column 
genre_combination_distribution = ghibli_data['Combined_Genres'].value_counts().reset_index()
genre_combination_distribution.columns = ['Combined_Genres', 'Film_Count']
```

In [23]:
# Load data
ghibli_data = pd.read_csv('Studio_Ghibli_Cleaned.csv')

# Create a new column combining the genre information
ghibli_data['Combined_Genres'] = ghibli_data[['Genre 1', 'Genre 2']].apply(
    lambda row: '-'.join(row.dropna().astype(str)), axis=1)

# View the combined genres column
print(ghibli_data[['Name', 'Combined_Genres']].head())

                              Name    Combined_Genres
0            When Marnie Was There    Animation-Drama
1  The Tale of The Princess Kaguya    Animation-Drama
2                   The Wind Rises    Drama-Animation
3            From Up on Poppy Hill    Animation-Drama
4     The Secret World of Arrietty  Fantasy-Animation


In [24]:
ghibli_data.to_csv('Studio_Ghibli_Final.csv', index=False)

In [25]:
# Count of each genre combination in the new Combined_Genres column 
genre_combination_distribution = ghibli_data['Combined_Genres'].value_counts().reset_index()
genre_combination_distribution.columns = ['Combined_Genres', 'Film_Count']

# Export to CSV
genre_combination_distribution.to_csv('Combined_Genres_Distribution.csv', index=False)

genre_combination_distribution

,Combined_Genres,Film_Count
0,Animation-Drama,7
1,Fantasy-Animation,3
2,Animation-Family,3
3,Adventure-Fantasy,3
4,Animation-Fantasy,2
5,Adventure-Animation,2
6,Drama-Animation,1
7,Family-Comedy,1
8,Fantasy-Adventure,1


In [113]:
%%jmd

### Aggregation of Revenue and Budget by Combined Genre:
Calculate average and total revenue and budget by combined genre.
```python
# Calculate average and total revenue and budget by combined genre:
combined_genre_financials = ghibli_data.groupby('Combined_Genres').agg(
    Average_Budget=('Budget', 'mean'),
    Total_Budget=('Budget', 'sum'),
    Average_Revenue=('Revenue', 'mean'),
    Total_Revenue=('Revenue', 'sum')
).reset_index()
```


### Aggregation of Revenue and Budget by Combined Genre:
Calculate average and total revenue and budget by combined genre.
```python
# Calculate average and total revenue and budget by combined genre:
combined_genre_financials = ghibli_data.groupby('Combined_Genres').agg(
    Average_Budget=('Budget', 'mean'),
    Total_Budget=('Budget', 'sum'),
    Average_Revenue=('Revenue', 'mean'),
    Total_Revenue=('Revenue', 'sum')
).reset_index()
```

In [107]:
# Calculate average and total revenue and budget by combined genre:
combined_genre_financials = ghibli_data.groupby('Combined_Genres').agg(
    Average_Budget=('Budget', 'mean'),
    Total_Budget=('Budget', 'sum'),
    Average_Revenue=('Revenue', 'mean'),
    Total_Revenue=('Revenue', 'sum')
).reset_index()

# Export to CSV
combined_genre_financials.to_csv('Combined_Genre_Financials.csv', index=False)

combined_genre_financials

,Combined_Genres,Average_Budget,Total_Budget,Average_Revenue,Total_Revenue
0,Adventure-Animation,6.500000e+06,1.300000e+07,2.236651e+09,4.473301e+09
1,Adventure-Fantasy,1.550000e+07,4.650000e+07,7.608171e+07,2.282451e+08
2,Animation-Drama,1.770714e+08,1.239500e+09,2.233488e+07,1.563441e+08
3,Animation-Family,1.546667e+07,4.640000e+07,1.488060e+08,4.464181e+08
4,Animation-Fantasy,2.800000e+07,5.600000e+07,1.355146e+08,2.710291e+08
5,Drama-Animation,3.000000e+07,3.000000e+07,1.179324e+08,1.179324e+08
6,Family-Comedy,9.200000e+06,9.200000e+06,3.410000e+07,3.410000e+07
7,Fantasy-Adventure,1.000000e+08,1.000000e+08,1.670000e+08,1.670000e+08
8,Fantasy-Animation,1.690000e+07,5.070000e+07,1.421767e+08,4.265302e+08


In [109]:
%%jmd

Since animation is the most frequently made genre, it might be worth filtering the data to only include films where the primary genre is animation and count each combination.
```python
# Filter data to include only films where the primary genre is Animation
animation_films = ghibli_data[ghibli_data['Genre 1'] == 'Animation']

# Count of each genre combination in Animation films
genre_combination_counts = animation_films['Combined_Genres'].value_counts().reset_index()
genre_combination_counts.columns = ['Animation_Combination', 'Count']
```


Since animation is the most frequently made genre, it might be worth filtering the data to only include films where the primary genre is animation and count each combination.
```python
# Filter data to include only films where the primary genre is Animation
animation_films = ghibli_data[ghibli_data['Genre 1'] == 'Animation']

# Count of each genre combination in Animation films
genre_combination_counts = animation_films['Combined_Genres'].value_counts().reset_index()
genre_combination_counts.columns = ['Animation_Combination', 'Count']
```

In [81]:
# Filter data to include only films where the primary genre is Animation
animation_films = ghibli_data[ghibli_data['Genre 1'] == 'Animation']

# Count of each genre combination in Animation films
genre_combination_counts = animation_films['Combined_Genres'].value_counts().reset_index()
genre_combination_counts.columns = ['Animation_Combination', 'Count']

# Export to CSV
genre_combination_counts.to_csv('Animation_Genre_Combinations.csv', index=False)

genre_combination_counts

,Animation_Combination,Count
0,Animation-Drama,7
1,Animation-Family,3
2,Animation-Fantasy,2


In [26]:
%%jmd

### Film Duration Analysis
Average and distribution of film durations by genre.
```python
# Average duration by genres
duration_by_genre = ghibli_data.groupby('Combined_Genres').agg(
    Average_Duration=('Duration_in_minutes', 'mean')
).reset_index()
```


### Film Duration Analysis
Average and distribution of film durations by genre.
```python
# Average duration by genres
duration_by_genre = ghibli_data.groupby('Combined_Genres').agg(
    Average_Duration=('Duration_in_minutes', 'mean')
).reset_index()
```

In [27]:
# Average duration by genres
duration_by_genre = ghibli_data.groupby('Combined_Genres').agg(
    Average_Duration=('Duration_in_minutes', 'mean')
).reset_index()

# Export to CSV
duration_by_genre.to_csv('Duration_by_Combined_Genre.csv', index=False)

duration_by_genre

,Combined_Genres,Average_Duration
0,Adventure-Animation,118.000000
1,Adventure-Fantasy,111.333333
2,Animation-Drama,103.142857
3,Animation-Family,110.666667
4,Animation-Fantasy,107.500000
5,Drama-Animation,126.000000
6,Family-Comedy,93.000000
7,Fantasy-Adventure,124.000000
8,Fantasy-Animation,99.666667


In [111]:
%%jmd

### Director Analysis
Films per director, categorized by genre.
```python
# Count of films by director
films_by_director = ghibli_data.groupby(['Director', 'Combined_Genres']).size().unstack(fill_value=0).reset_index()
```


### Director Analysis
Films per director, categorized by genre.
```python
# Count of films by director
films_by_director = ghibli_data.groupby(['Director', 'Combined_Genres']).size().unstack(fill_value=0).reset_index()
```

In [79]:
# Count of films by director
films_by_director = ghibli_data.groupby(['Director', 'Combined_Genres']).size().unstack(fill_value=0).reset_index()

# Export to CSV
films_by_director.to_csv('Films_by_Director.csv', index=False)

films_by_director

Combined_Genres,Director,Adventure-Animation,Adventure-Fantasy,Animation-Drama,Animation-Family,Animation-Fantasy,Drama-Animation,Family-Comedy,Fantasy-Adventure,Fantasy-Animation
0,Goro Miyazaki,0,0,1,0,1,0,0,0,0
1,Hayao Miyazaki,1,2,0,2,1,1,1,1,2
2,Hiromasa Yonebayashi,0,0,1,0,0,0,0,0,1
3,Hiroyuki Morita,0,1,0,0,0,0,0,0,0
4,Isao Takahata,1,0,3,1,0,0,0,0,0
5,Tomomi Mochizuki,0,0,1,0,0,0,0,0,0
6,Yoshifumi Kondo,0,0,1,0,0,0,0,0,0


In [4]:
%%jmd

___

# Studio Ghibli Dashboard

Below is an image of the dashboard. To view and interact with the dashboard, I've provided a link to it on Tableau Public. 

![Dashboard](dashboard.png)

View the interactive dashboard on [Tableau Public](https://public.tableau.com/app/profile/brittany.everette/viz/StudioGhibliAnalyzingGenrePopularity/Dashboard1#1)


___

# Studio Ghibli Dashboard

Below is an image of the dashboard. To view and interact with the dashboard, I've provided a link to it on Tableau Public. 

![Dashboard](dashboard.png)

View the interactive dashboard on [Tableau Public](https://public.tableau.com/app/profile/brittany.everette/viz/StudioGhibliAnalyzingGenrePopularity/Dashboard1#1)

In [33]:
%%jmd

___


# Final Findings and Recommendations

## Findings

The analysis of Studio Ghibli's film catalog provides a detailed look into genre popularity and their financial outcomes:

- **Distribution of Genre Combinations**: The most frequently produced genre is Animation-Drama, with seven films to its credit. This consistency indicates a solid audience base and our studio's dedication to this genre.

- **Aggregation of Revenue and Budget by Combined Genre**: Adventure-Animation, represented by just two films, has yielded the highest total revenue, surpassing 4.47 billion. This exceptional financial success highlights its potential for yielding substantial returns.

- **Film Duration Analysis**: The extended duration of films in genres like Adventure-Animation and Fantasy-Adventure generally correlates with strong market acceptance and sustained audience interest.

- **Director Analysis**: The versatile Hayao Miyazaki has contributed significantly across various high-revenue genres, reinforcing his critical role in the studio's successful outputs.

The analysis supports two primary success metrics:
1. **Highest Revenue**: Adventure-Animation, despite its limited production, stands out for its incredible revenue potential per film.
2. **Consistent Production**: Animation-Drama has been produced most consistently, affirming its role as a fundamental genre that continuously attracts viewers.

## Recommendations

Reflecting on the findings, I recommend the following strategies to optimize both financial success and audience engagement:

1. **Increase Production of Adventure-Animation**: Considering its impressive revenue achievements, investing in more Adventure-Animation films could tap into this genre's lucrative potential.

2. **Maintain Focus on Animation-Drama**: The steady production of Animation-Drama films shows its enduring appeal. Investing in this genre could include pushing the boundaries of narrative depth or introducing innovative storytelling techniques to refresh its appeal.

3. **Leverage Directorial Strengths**: Assigning projects within these successful genres to experienced directors like Hayao Miyazaki can maximize their success potential, given his track record in enhancing film performance.

4. **Refine Audience Engagement Strategies**: Adjusting film lengths to meet genre-specific audience expectations could further improve engagement and satisfaction levels.

Adopting these strategies can ensure that future projects will not only resonate with audiences but also continue to uphold Studio Ghibli's legacy in the competitive landscape of animation cinema.

___


___


# Final Findings and Recommendations

## Findings

The analysis of Studio Ghibli's film catalog provides a detailed look into genre popularity and their financial outcomes:

- **Distribution of Genre Combinations**: The most frequently produced genre is Animation-Drama, with seven films to its credit. This consistency indicates a solid audience base and our studio's dedication to this genre.

- **Aggregation of Revenue and Budget by Combined Genre**: Adventure-Animation, represented by just two films, has yielded the highest total revenue, surpassing 4.47 billion. This exceptional financial success highlights its potential for yielding substantial returns.

- **Film Duration Analysis**: The extended duration of films in genres like Adventure-Animation and Fantasy-Adventure generally correlates with strong market acceptance and sustained audience interest.

- **Director Analysis**: The versatile Hayao Miyazaki has contributed significantly across various high-revenue genres, reinforcing his critical role in the studio's successful outputs.

The analysis supports two primary success metrics:
1. **Highest Revenue**: Adventure-Animation, despite its limited production, stands out for its incredible revenue potential per film.
2. **Consistent Production**: Animation-Drama has been produced most consistently, affirming its role as a fundamental genre that continuously attracts viewers.

## Recommendations

Reflecting on the findings, I recommend the following strategies to optimize both financial success and audience engagement:

1. **Increase Production of Adventure-Animation**: Considering its impressive revenue achievements, investing in more Adventure-Animation films could tap into this genre's lucrative potential.

2. **Maintain Focus on Animation-Drama**: The steady production of Animation-Drama films shows its enduring appeal. Investing in this genre could include pushing the boundaries of narrative depth or introducing innovative storytelling techniques to refresh its appeal.

3. **Leverage Directorial Strengths**: Assigning projects within these successful genres to experienced directors like Hayao Miyazaki can maximize their success potential, given his track record in enhancing film performance.

4. **Refine Audience Engagement Strategies**: Adjusting film lengths to meet genre-specific audience expectations could further improve engagement and satisfaction levels.

Adopting these strategies can ensure that future projects will not only resonate with audiences but also continue to uphold Studio Ghibli's legacy in the competitive landscape of animation cinema.

___

In [32]:
%%jmd

# Acknowledgments & References

- The dataset is available on Kaggle and can be downloaded using the following link:
    - [Studio Ghibli Dataset on Kaggle](https://www.kaggle.com/datasets/shruthiiiee/studio-ghibli-dataset)

- This project is licensed under the [Apache License, Version 2.0.](http://www.apache.org/licenses/LICENSE-2.0)

- The dashboard was created using [Tableau Public](https://public.tableau.com/). 

- The background of the dashboard was created and designed myself using [Canva](https://www.canva.com).

- The Studio Ghibli pngs in the dashboard are from [PNGEgg](https://www.pngegg.com/) and are free to use. 

___

## Contact Information

- Email: [phelpsbp@gmail.com](mailto:phelpsbp@gmail.com)
- LinkedIn: [Brittany Phelps](https://www.linkedin.com/in/brittany-everette/)
- Github: [phelpsbp](https://github.com/phelpsbp)

I'm usually available for a chat on weekdays from 9 AM to 5 PM. Looking forward to hearing from you!


# Acknowledgments & References

- The dataset is available on Kaggle and can be downloaded using the following link:
    - [Studio Ghibli Dataset on Kaggle](https://www.kaggle.com/datasets/shruthiiiee/studio-ghibli-dataset)

- This project is licensed under the [Apache License, Version 2.0.](http://www.apache.org/licenses/LICENSE-2.0)

- The dashboard was created using [Tableau Public](https://public.tableau.com/). 

- The background of the dashboard was created and designed myself using [Canva](https://www.canva.com).

- The Studio Ghibli pngs in the dashboard are from [PNGEgg](https://www.pngegg.com/) and are free to use. 

___

## Contact Information

- Email: [phelpsbp@gmail.com](mailto:phelpsbp@gmail.com)
- LinkedIn: [Brittany Phelps](https://www.linkedin.com/in/brittany-everette/)
- Github: [phelpsbp](https://github.com/phelpsbp)

I'm usually available for a chat on weekdays from 9 AM to 5 PM. Looking forward to hearing from you!

In [8]:
!jupyter nbconvert --to html --template pj --embed-images "Studio Ghibli Genre Popularity.ipynb"

[NbConvertApp] Converting notebook Studio Ghibli Genre Popularity.ipynb to html
[NbConvertApp] Writing 6617480 bytes to Studio Ghibli Genre Popularity.html


In [37]:
pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.
